In [ ]:
!pip install git+https://github.com/huggingface/transformers.git sentencepiece

In [2]:
from transformers import SeamlessM4Tv2Model
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.72k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/211k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.24G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

In [3]:
text_inputs = processor(text="Hello, how are you today? I am a therapist.", src_lang="eng", return_tensors="pt").to(device)
output_tokens = model.generate(**text_inputs, tgt_lang="pes", generate_speech=False)
translated_text_from_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
translated_text_from_text

'سلام، امروز حالتون چطوره؟ من یک درمانگر هستم.'

In [ ]:
# import gdown
# url = "https://drive.google.com/file/d/1frPNIzvukPRc2fguIuOCT1WBLT5qJRIS/view?usp=sharing"
# output = "text.csv"
# gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1frPNIzvukPRc2fguIuOCT1WBLT5qJRIS
To: /content/text.csv
100%|██████████| 14.0M/14.0M [00:00<00:00, 45.7MB/s]


'text.csv'

In [4]:
import pandas as pd

df = pd.read_csv('combined_transcripts.csv')
# df = df.drop(columns=['translated_column', 'translated_context', 'translated_response'])
df.head()

,ID,Type,Utterance,Dialog_Act
0,163_0,T,"Hey Michelle, how are you doing today?",gt
1,163_1,P,"Umm I am kind of doing, I'm feeling a little b...",gt
2,163_2,T,stressed?,irq
3,163_3,P,"Yeah, I feel like I have a lot on my plate. I ...",id
4,163_4,T,What kind of test?,irq


In [5]:
from tqdm import tqdm
import pandas as pd
from functools import lru_cache

# Hypothetical translation function with caching
@lru_cache(maxsize=None)
def translate(text):
    text_inputs = processor(text=text, src_lang="eng", return_tensors="pt").to(device)
    output_tokens = model.generate(**text_inputs, tgt_lang="pes", generate_speech=False)
    translated_text_from_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
    return translated_text_from_text

# Apply translation with tqdm progress bar
tqdm.pandas()
# df = df.head(1000)

# Translate context and response and store them in new columns
df['translated_Utterance'] = df['Utterance'].progress_apply(translate)

# Save the updated DataFrame to a new CSV file
df.to_csv('combined_transcripts_translated.csv', index=False)

# Print the results
print("Updated DataFrame with translations:")
print(df.head())


100%|██████████| 479/479 [09:29<00:00,  1.19s/it]

Updated DataFrame with translations:
      ID Type                                          Utterance Dialog_Act  \
0  163_0    T             Hey Michelle, how are you doing today?         gt   
1  163_1    P  Umm I am kind of doing, I'm feeling a little b...         gt   
2  163_2    T                                          stressed?        irq   
3  163_3    P  Yeah, I feel like I have a lot on my plate. I ...         id   
4  163_4    T                                 What kind of test?        irq   

                                translated_Utterance  
0                       سلام میشل، امروز حالت چطوره؟  
1  امم، من دارم کار می کنم، احساس می کنم کمی استر...  
2                                        استرس داری؟  
3  آره، احساس ميکنم که خيلي چيزا روي سر من هست، م...  
4                                     چه نوع آزمونی؟  


In [ ]:
df.head()

,Context,Response,translated_context,translated_response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",من در حال گذراندن برخی از مسائل با احساسات و خ...,اگر همه فکر می کنند که بی ارزش هستید، پس شاید ...
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",من در حال گذراندن برخی از مسائل با احساسات و خ...,در واقع، بیشتر افراد، اگر نه همه، تا حدودی در ...
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,من در حال گذراندن برخی از مسائل با احساسات و خ...,اولین چیزی که من پیشنهاد می کنم این است که به ...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,من در حال گذراندن برخی از مسائل با احساسات و خ...,درمان با خودآگاهی (CBT) کمک می کند تا آگاهی به...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,من در حال گذراندن برخی از مسائل با احساسات و خ...,اول می خواهم به شما بگویم که در احساسات خود تن...
